# Face Detection and Feature Visualization
## Excercise 3.

### OpenCV 및 Detector 정보 파싱하기

In [ ]:
"""
*Uncomment if running on colab* 
Set Runtime -> Change runtime type -> Under Hardware Accelerator select GPU in Google Colab 
"""
# !git clone https://github.com/hoihoic/OpenCV_face_detection_exercise.git
# !mv OpenCV_face_detection_exercise/* ./

import cv2
import time
import xml.etree.ElementTree as ET

xml_path = r'./haarcascade_frontalcatface_extended.xml'
image_path = r'./00204_Mask.jpg'

# Parsing xml for face detector
tree = ET.parse(xml_path)
root = tree.getroot()

haarcascade = [] 
feat_list = []

### 얼굴 검출 및 관심 영역 추출

In [ ]:
# Load an image
image = cv2.imread(image_path)

# Create and Load the detector
face_cascade = cv2.CascadeClassifier()
face_cascade.load(xml_path)

# Convert RGB to gray for input
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(image_gray, scaleFactor=1.02)
x, y, w, h = faces[0]
image = image[y:y+h, x:x+w, :]

w_f = int(root.findall('.//width')[0].text)
h_f = int(root.findall('.//height')[0].text)

h_ratio, w_ratio = h / h_f, w / w_f

### Weak classifier 정보 추출

In [ ]:
# Get rectangle information from xml tree
for rects in root.iter('rects'):
    feat_dict = {'rect': [], 'val': []}
    for i in rects.iter('_'):
        vals = i.text.split(" ")
        x = int(vals[10])
        y = int(vals[11])
        w = int(vals[12])
        h = int(vals[13])
        v = float(vals[14])
        
        feat_dict['rect'] += [[x, y, w, h]]
        feat_dict['val'] += [v]
        
    feat_list += [feat_dict]

### Feature Visualization

In [ ]:
# white region: positive weight
# black region: negative region
def draw_feature(image, feat, w_ratio, h_ratio):
    img = image.copy()
    
    rects = feat['rect']
    val = feat['val']
    
    for i, r in enumerate(rects):
        x, y, w, h = r
        v = val[i]
        start = (int(x*w_ratio), int(y*h_ratio))
        end = (int((x+w)*w_ratio), int((y+h)*h_ratio))
        
        color = (255, 255, 255) if v > 0 else (0, 0, 0)

        img = cv2.rectangle(img, start, end, color, -1)
        
    cv2.imshow('image with feat', img)
    key = cv2.waitKey(3000)
    if key == 27:
        exit()

# find every stages and classifiers
for stage, child in enumerate(root.iter('weakClassifiers')):
    for cchild in child.iter('internalNodes'):
        n_value = cchild.text
        n_value = n_value.split(' ')
        idx_feat = int(n_value[14])
        
        haarcascade_stage = {'rect': feat_list[idx_feat]['rect'], 'val':feat_list[idx_feat]['val']}
        haarcascade += [haarcascade_stage]
        
        draw_feature(image, haarcascade_stage, w_ratio, h_ratio)       